In [1]:
# load data from csv
# path for input and target data tables

diagnosis_path = '../data/diagnosis_hadm.csv'
discharge_path ='../data/discharge.csv'
edstays_path = '../data/edstays.csv'
radiology_path = '../data/radiology.csv'
triage_path = '../data/triage.csv'
target_path = '../data/discharge_target.csv'
discharge_sections_path = '../data/discharge_sections.csv'
radiology_sections_path = '../data/radiology_sections.csv'

import pandas as pd
import re
import os

# read data
diagnosis_df = pd.read_csv(diagnosis_path, keep_default_na=False)
discharge_df = pd.read_csv(discharge_path, keep_default_na=False)
edstays_df = pd.read_csv(edstays_path, keep_default_na=False)
radiology_df = pd.read_csv(radiology_path, keep_default_na=False)
triage_df = pd.read_csv(triage_path, keep_default_na=False)
target_df = pd.read_csv(target_path, keep_default_na=False)

discharge_sections_df = pd.read_csv(discharge_sections_path, keep_default_na=False)
radiology_sections_df = pd.read_csv(radiology_sections_path, keep_default_na=False)

In [13]:
discharge_sections_df.head(2)


,Unnamed: 0,subject_id,hadm_id,CC,Service,Major Surgical Procedure,HPI,PMH,SOC,FH,...,Problem List,Physical Exam,Medication Lists,Pertinent Results,BHC,Transitional Issues,Disposition,Discharge Instructions,Followup Instructions,Discharge Diagnosis
0,0,10001884,24962904,Shortness of Breath,"MEDICINE\n \nAllergies: \nIV Dye, Iodine Conta...",N/A,Ms. ___ is a ___ female with history of \nCOPD...,- COPD/Asthma on home 2L O2\n- Atypical Chest ...,___,Mother with asthma and hypertension. Father wi...,...,,ADMISSION PHYSICAL EXAM:\n====================...,is accurate and complete.\n1. Acetaminophen 32...,ADMISSION LABS: \n=========================\n_...,Ms. ___ is a ___ female with history of \nCOPD...,==========================\n[] For pt's contin...,Extended Care\n \nFacility:\n___\n \n___ Diagn...,"Dear Ms. ___,\n\nYou were admitted to ___ afte...",,PRIMARY:\nCOPD Exacerbation\n\nSECONDARY:\nAfi...
1,1,10003019,22774359,fever,MEDICINE\n \nAllergies: \nRagweed / morphine /...,none,Mr ___ is a ___ with h/o stage IV Hodgkins c1d...,"1. Sarcoidosis, dx skin bx: intestinal & pulmo...",___,"Mother: ___, cardiac disease. \nFather: diver...",...,,ADMISSION EXAM\nVitals: 124/67 on neosynephrin...,The Preadmission Medication list is accurate a...,ADMISSION LABS\n___ 10:40AM BLOOD WBC-0.2* RBC...,___ male with h/o Hodgkin's lymphoma C1D17 ABV...,,Home With Service\n \nFacility:\n___\n \nDisch...,"Dear Mr. ___,\n\nIt has been our pleasure to b...",,"Primary Diagnosis\nNeutropenic Fever, no sourc..."


In [ ]:
import pandas as pd
import torch
from transformers import LlamaTokenizer, LlamaForCausalLM
from transformers import pipeline, AutoTokenizer

def generate_summary_with_pipeline(final_prompt, tokenizer):
    context_length = len(tokenizer.tokenize(final_prompt))
    max_input_length = min(1024 + context_length, tokenizer.model_max_length)
    
    # Initialize the pipeline with the Llama-3 model
    print("\nQ&A with model: Llama-3")
    llama_pipe = pipeline('text-generation', model='meta-llama/Meta-Llama-3-8B')

    # Generate the response using the model
    generated_text = llama_pipe(final_prompt, max_length=max_input_length, num_return_sequences=1)
    # Output the generated text
    generated_text[0]['generated_text'][context_length:]
    return generated_text[0]['generated_text']

In [ ]:


# Load the model and tokenizer
# tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
# model = LlamaForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
model = LlamaForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B")

In [ ]:
# prepare prompt

hadm_id = 24962904
# Specify the columns to use
df = discharge_sections_df
discharge_cols = ['HPI']
# radiology_cols = ['EXAMINATION', 'INDICATION', 'IMPRESSION']
# Filter DataFrames for the given HADM ID
discharge_info = df[df['hadm_id'] == hadm_id][discharge_cols].fillna('').agg(' '.join, axis=1).values[0]
# radiology_info = radiology_df[radiology_df['hadm_id'] == hadm_id][radiology_cols].fillna('').agg(' '.join, axis=1).values[0]


# Combine all relevant information into one context string
context = f"The history of present illness is: {discharge_info}"
question = "Make a list of medical problems, associated medical workup and associated treatments. Limit the list to 12 items"

# Prepare the final prompt for the model
final_prompt = f"Context: This conversation is about clinical notes during a hospital stay. {context} \n\nQuestion: Based on the information provided, {question} \n\nAnswer:"
ans = generate_summary_with_pipeline(final_prompt, tokenizer)
print(ans)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]


Q&A with model: Llama-3


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Context: This conversation is about clinical notes during a hospital stay. The history of present illness is: Ms. ___ is a ___ female with history of 
COPD on home O2, atrial fibrillation on apixaban, hypertension, 
CAD, and hyperlipidemia who presents with shortness of breath, 
cough, and wheezing for one day.

The patient reports shortness of breath, increased cough 
productive of ___ red-flected sputum, and wheezing since 
yesterday evening.  She has been using albuterol IH more 
frequently (___) with ipratropium nebs every 4 hours with 
minimal relief. She had to increase her O2 flow up to 4L without 
significant improvement. She was currently taking 10mg of 
prednisone. She has also been taking tiotropium IH, 
theophylline, advair IH at home as prescribed. She denies sick 
contacts. She quit smoking approximately 1 month ago.

She reports an episode of chest pain in waiting room while 
sitting down, non-exertional, resolved after 2 minutes. She 
denies fever/chills, abdominal pain

In [14]:
print(ans)

Context: This question is about clinical notes during a hospital stay. The history of present illness is: Ms. ___ is a ___ female with history of 
COPD on home O2, atrial fibrillation on apixaban, hypertension, 
CAD, and hyperlipidemia who presents with shortness of breath, 
cough, and wheezing for one day.

The patient reports shortness of breath, increased cough 
productive of ___ red-flected sputum, and wheezing since 
yesterday evening.  She has been using albuterol IH more 
frequently (___) with ipratropium nebs every 4 hours with 
minimal relief. She had to increase her O2 flow up to 4L without 
significant improvement. She was currently taking 10mg of 
prednisone. She has also been taking tiotropium IH, 
theophylline, advair IH at home as prescribed. She denies sick 
contacts. She quit smoking approximately 1 month ago.

She reports an episode of chest pain in waiting room while 
sitting down, non-exertional, resolved after 2 minutes. She 
denies fever/chills, abdominal pain, na

In [18]:
import langchain
import pandas as pd
import torch
import os
from transformers import LlamaTokenizer, LlamaForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer

from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

template = """

This question is about clinical notes from an electronic health record that were created during a hospital stay.

Use the following pieces of context from this patient's record to answer the question at the end.

Use three sentences maximum, medical terminology or abreviations, and keep the answer as concise as possible.

Use the active voice, and speak directly to the reader using concise language.
{context}

Question: {question}

Answer:

"""
hadm_id = 24962904
# Specify the columns to use
df = discharge_sections_df
discharge_cols = ['HPI']
discharge_info = df[df['hadm_id'] == hadm_id][discharge_cols].fillna('').agg(' '.join, axis=1).values[0]


# Combine all relevant information into one context string
context = f"The history of present illness is: {discharge_info}"
question = "What were the patient's medical problems, associated medical workup and associated treatments?"


QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

print(QA_CHAIN_PROMPT)
print(template)



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

input_variables=['context', 'question'] template="\n\nThis question is about clinical notes from an electronic health record that were created during a hospital stay.\n\nUse the following pieces of context from this patient's record to answer the question at the end.\n\nUse three sentences maximum, medical terminology or abreviations, and keep the answer as concise as possible.\n\nUse the active voice, and speak directly to the reader using concise language.\n{context}\n\nQuestion: {question}\n\nAnswer:\n\n"


This question is about clinical notes from an electronic health record that were created during a hospital stay.

Use the following pieces of context from this patient's record to answer the question at the end.

Use three sentences maximum, medical terminology or abreviations, and keep the answer as concise as possible.

Use the active voice, and speak directly to the reader using concise language.
{context}

Question: {question}

Answer:




In [5]:
import sys
!{sys.executable} -m pip install torch
!{sys.executable} -m pip install transformers
!{sys.executable} -m pip install protobuf
!{sys.executable} -m pip install sentencepiece
!{sys.executable} -m pip install langchain

  Using cached langchain-0.1.16-py3-none-any.whl.metadata (13 kB)
  Using cached SQLAlchemy-2.0.29-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
  Using cached aiohttp-3.9.5-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.5 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_community-0.0.34-py3-none-any.whl.metadata (8.5 kB)
  Using cached langchain_text_splitters-0.0.1-py3-none-any.whl.metadata (2.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 7.8 MB/s eta 0:00:00
  Using cached tenacity-8.2.3-py3-none-any.whl.metadata (1.0 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached frozenlist-1.4.1-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multidict-6.0.5-cp39-cp39-manylinux_2_17_x86_64.manyl

In [10]:
import torch
from transformers import LlamaTokenizer, LlamaForCausalLM
import pandas as pd

# Load the model and tokenizer
tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
model = LlamaForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

# def query_with_llama(question, dataframes, column_lists, hadm_id, output_column):
    
#     context_data = ""

#     # Iterate through each DataFrame and its corresponding columns to create the context
#     for df, cols in zip(dataframes, column_lists):
#         # Filter the dataframe for the specific hadm_id
#         target_df = df[df['hadm_id'] == hadm_id]
#         if not target_df.empty:
#             # Create a context string from the specified columns
#             context_part = target_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep=' ')
#             context_data += " " + context_part
    
#     if not context_data:
#         dataframes[0].loc[dataframes[0]['hadm_id'] == hadm_id, output_column] = "No records found for the given HADM ID."
#         return dataframes[0]

#     # Combine the question with the aggregated context data
#     inputs = tokenizer(question + " " + context_data, return_tensors="pt", truncation=True, max_length=1024)

#     # Generate the response using the model
#     with torch.no_grad():  # Disable gradient calculation for inference
#         outputs = model.generate(**inputs, max_length=1024)
    
#     answer = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    
#     # Append the answer to the first DataFrame
#     dataframes[0].loc[dataframes[0]['hadm_id'] == hadm_id, output_column] = answer
#     return dataframes[0]

# # Example usage
# question = "Summarize the radiological tests and findings"
# hadm_id = 24962904  # Specify the HADM ID you want to query
# dfs = [discharge_sections_df, radiology_sections_df]  # List of DataFrames
# relevant_cols = [['Pertinent Results'], ['EXAMINATION','INDICATION','IMPRESSION']]  # List of column lists for each DataFrame
# output_col_name = 'radiology tests summary'  # Specify the name of the new column

# # Assuming df1 is the primary DataFrame where the output should be stored
# df1 = query_with_llama(question, dfs, relevant_cols, hadm_id, output_col_name)
# print(df1.loc[df1['hadm_id'] == hadm_id, output_col_name])



ImportError: 
LlamaTokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


In [4]:
question = "Given the following information from a clinical note, what are the patient's major medical problems:"

# get the relevant data from the patient chart
hadm_id = 24962904  # Specify the HADM ID you want to query
df = [discharge_sections_df, radiology_sections_df]  # List of DataFrames
relevant_cols = [['HPI'],['CC'] ]

# combine context_text in question to make prompt

# # Combine the question with the aggregated context data
# prompt_text = ""
# prompt_text = question + " " + context_data

# # Encode the prompt text
# input_ids = tokenizer.encode(prompt_text, return_tensors='pt')

# runnable = prompt | model | StrOutputParser()


NameError: name 'dataframes' is not defined